In [ ]:


def duckduckgo_search(query, max_results=5):
    from duckduckgo_search import DDGS

    results = []
    try:
        with DDGS() as ddgs:
            search_results = ddgs.text(query, max_results=max_results)
            for r in search_results:
                results.append({
                    "title": r.get("title", ""),
                    "url": r.get("url", ""), 
                    "snippet": r.get("body", "")
                })
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return [{"title": "Error occurred", "url": "", "snippet": str(e)}]
    
    if not results:
        return [{"title": "No results found", "url": "", "snippet": ""}]
    
    return results



results = duckduckgo_search("Current price of rtx 3060")
print(results)

[{'title': 'GeForce RTX 3060 GPUs / Video Graphics Cards | Newegg.com', 'url': '', 'snippet': 'Shop GeForce RTX 3060 GPUs / Video Graphics Cards on Newegg.com. Watch for amazing deals and get great pricing.'}, {'title': 'RTX 3060 Price Tracker US - Jan 2025 - bestvaluegpu.com', 'url': '', 'snippet': 'Nvidia RTX 3060 price tracker provides current and historical graphics card pricing data in the US market. To help you make an informed decision. ... Nvidia RTX 3060 price is $279 on Amazon currently. Used price is around $190 on ebay. The release price (MSRP) in US was $329 at launch. MSI Gaming GeForce RTX 3060'}, {'title': 'NVIDIA GeForce RTX 3060 GPUs / Video Graphics Cards', 'url': '', 'snippet': 'Find low everyday prices and buy online for delivery or in-store pick-up ... Current Deals. On Sale; Clearance; Price. to. $250 - $499.99; Discount. All Discounted Items; Customer Rating. Top-Rated; 4 & Up; ... NVIDIA GeForce RTX 3060 Ventus 3X 12G OC - 12GB GDDR6 - PCI Express 4.0 - Graphic